In [24]:
import json
import pandas as pd
import numpy as np
import random

#### Extract Tables ####

In [25]:
# Load data from JSON file
with open('/content/tables_updated.json') as file:
    data = json.load(file)

# Extracted data for tables
extracted_data = []

# Counter for generating table IDs
table_id_counter = 0

# Extract table information
for table_name, table_info in data.items():
    # Increment table ID counter
    table_id_counter += 1

    record = {
        "Table ID": table_id_counter,
        "Db Name": data[table_name].get("Name of DB", ""),
        "Table Name": table_name,
        "Display Text": table_info.get("Display Text", ""),
        "Owner": table_info.get("Owner", ""),
        "Status": table_info.get("Status", ""),
        "Total Size": table_info.get("Total Size", "0").replace('N/A', '0'),
        "Raw Data Size": table_info.get("Raw Data Size", "0").replace('N/A', '0'),
        "Number of Rows": table_info.get("Number of Rows", "0").replace('N/A', '0'),
        "Number of Files": table_info.get("Number of Files", "0").replace('N/A', '0'),
        "Number of Attributes": table_info.get("Number of Attributes", "0"),
        "Classification Names": table_info.get("Classification Names", []),
        "Motifs": table_info.get("Motifs", []),
        "Valid Db Name": table_info.get("Valid_DbName", ""),
        "Valid Table Name": table_info.get("Valid_TableName", ""),
        "Unique GUID": table_info.get("Unique_GUID", ""),
        "Future CreateTime": table_info.get("Future_CreateTime", ""),
        "Future lastAccessTime": table_info.get("Future_lastAccessTime", ""),
        "LastAccessBeforeCreate": table_info.get("LastAccessBeforeCreate", ""),
        "label": table_info.get("label", ""),
    }

    record["Create Time"] = pd.to_datetime(table_info.get("Create Time", ""))
    record["lastAccessTime"] = pd.to_datetime(table_info.get("lastAccessTime", ""))

    # Convert numeric values to numeric type
    exclude_keys = ["Table Name", "Display Text", "Db Name", "Owner", "Status", "Classification Names","Motifs","label"]
    for key, value in record.items():
        if key not in exclude_keys and key not in ["Create Time", "lastAccessTime"]:
            record[key] = pd.to_numeric(value, errors='coerce')

    extracted_data.append(record)

In [26]:
df_out = pd.DataFrame(extracted_data)
df_out.head()

,Table ID,Db Name,Table Name,Display Text,Owner,Status,Total Size,Raw Data Size,Number of Rows,Number of Files,...,Motifs,Valid Db Name,Valid Table Name,Unique GUID,Future CreateTime,Future lastAccessTime,LastAccessBeforeCreate,label,Create Time,lastAccessTime
0,1,default bcp,analyse_risque,analyse_risque,azureuser,ACTIVE,-4490,4440,-50,1,...,"[Invalid Database Name, Invalid Table Name, Cr...",False,False,True,False,False,False,0,2027-05-02 14:28:14,2024-05-02 14:28:14
1,2,default,annonces_data,annonces_data,azureuser,ACTIVE,19084,18984,100,1,...,"[Invalid Table Name, Invalid Number of Attribu...",True,False,True,False,False,False,0,2021-05-02 14:31:13,2024-05-02 14:31:13
2,3,default,assistance_technique,assistance_technique,azureuser,ACTIVE,4891,4841,50,1,...,[No Classifications],True,True,True,False,False,False,0,2024-05-02 11:15:44,2024-05-02 11:15:44
3,4,default,alertes_ops_frauduleuses,alertes_ops_frauduleuses,azureuser,ACTIVE,2800,2750,50,1,...,[No Classifications],True,True,True,False,False,False,0,2024-05-02 11:10:34,2024-05-02 11:10:34
4,5,default,analyse_portefeuille,analyse_portefeuille,azureuser,ACTIVE,1833,1783,50,1,...,[No Classifications],True,True,True,False,False,False,0,2024-05-03 22:37:54,2024-05-03 22:37:54


In [27]:
# Expanding the 'Motifs' and 'Classifications' lists into separate columns
motif_columns = df_out['Motifs'].apply(pd.Series).rename(columns=lambda x: f'Motif_{x+1}')
classification_columns = df_out['Classification Names'].apply(pd.Series).rename(columns=lambda x: f'Classification_{x+1}')

# Concatenate the new columns with the original DataFrame
df_out = pd.concat([df_out.drop(['Motifs', 'Classification Names'], axis=1), motif_columns, classification_columns], axis=1)

# Now you can save this DataFrame to a CSV or JSON, or process it further


In [28]:
df_out.head()

,Table ID,Db Name,Table Name,Display Text,Owner,Status,Total Size,Raw Data Size,Number of Rows,Number of Files,...,lastAccessTime,Motif_1,Motif_2,Motif_3,Motif_4,Motif_5,Motif_6,Motif_7,Classification_1,Classification_2
0,1,default bcp,analyse_risque,analyse_risque,azureuser,ACTIVE,-4490,4440,-50,1,...,2024-05-02 14:28:14,Invalid Database Name,Invalid Table Name,Create Time '2027-05-02 14:28:14' is in the fu...,lastAccessTime '2024-05-02 14:28:14' is before...,Total Size has an invalid value: -4490.,Number of Rows has an invalid value: -50.,No Classifications,NaN,NaN
1,2,default,annonces_data,annonces_data,azureuser,ACTIVE,19084,18984,100,1,...,2024-05-02 14:31:13,Invalid Table Name,Invalid Number of Attributes,NaN,NaN,NaN,NaN,NaN,Secret,NaN
2,3,default,assistance_technique,assistance_technique,azureuser,ACTIVE,4891,4841,50,1,...,2024-05-02 11:15:44,No Classifications,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,default,alertes_ops_frauduleuses,alertes_ops_frauduleuses,azureuser,ACTIVE,2800,2750,50,1,...,2024-05-02 11:10:34,No Classifications,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,default,analyse_portefeuille,analyse_portefeuille,azureuser,ACTIVE,1833,1783,50,1,...,2024-05-03 22:37:54,No Classifications,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
df_out.to_csv('/content/data_anomaly_v2.csv', index=False)